## 1. 사용할 패키지 불러오기

In [2]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Reshape
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from fastprogress import master_bar, progress_bar

2023-12-20 00:06:26.434281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 00:06:26.571732: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 00:06:27.141614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-12-20 00:06:27.141701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot o

## 2. 데이터 불러오기

In [3]:
data = pd.read_csv("data.csv")
data.head()

,시가(포인트),고가(포인트),저가(포인트),종가(포인트),종가베이시스,코스피200시가총액,거래량(계약),거래대금(천원),잔존일수(일),미결제약정(계약),...,국가별수출액 합계(천달러),WTI선물_종가,브렌트유선물_종가,천연가스선물_종가,금선물_종가,은선물_종가,구리선물_종가,MACD,RSI,Date
0,238.30,239.25,235.60,238.30,1.58,7.939651e+14,154511.0,1.836201e+10,88.0,79906.0,...,30998134.0,80.570000,76.980003,6.653,715.799988,12.739,3.4175,0.828727,52.630885,2007-09-17
1,238.00,238.60,233.15,233.60,1.18,7.794642e+14,199834.0,2.359650e+10,87.0,79909.0,...,30998134.0,81.510002,77.589996,6.568,715.799988,12.767,3.4425,0.724943,47.740409,2007-09-18
2,242.55,245.00,241.45,245.00,3.92,8.076760e+14,162505.0,1.973762e+10,86.0,84491.0,...,30998134.0,81.930000,78.470001,6.180,722.000000,12.956,3.5675,1.544772,57.947348,2007-09-19
3,244.00,246.35,243.25,244.65,2.23,8.119758e+14,170882.0,2.089486e+10,85.0,84638.0,...,30998134.0,83.320000,79.089996,6.008,732.400024,13.321,3.5740,2.141563,57.575543,2007-09-20
4,244.65,247.10,244.25,245.80,2.35,8.155575e+14,150500.0,1.847012e+10,84.0,81140.0,...,30998134.0,81.620003,79.300003,6.080,731.400024,13.474,3.5725,2.676467,58.517352,2007-09-21


## 3. 데이터 전처리

#### - 증감 여부 추가

In [4]:
data['Increasing'] = 0
for i in range(1, len(data)):
    if data['종가(포인트)'][i] > data['종가(포인트)'][i-1]:
        data['Increasing'][i] = 1


/tmp/ipykernel_27313/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_27313/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_27313/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Increasing'][i] = 1
/tmp/ipykernel_27313/1910206775.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

#### - Input 과 Target 정의

In [5]:
Input_columns = ['시가(포인트)', '고가(포인트)', '저가(포인트)', '종가베이시스', '코스피200시가총액',
       '거래량(계약)', '거래대금(천원)', '잔존일수(일)', '미결제약정(계약)',
       '주가지수선물-Futures(SP합)-순매수대금(개인)(백만원)',
       '주가지수선물-Futures(SP합)-순매수대금(외국인)(백만원)',
       '주가지수선물-Futures(SP합)-순매수대금(기관계)(백만원)', '시장금리:CD유통수익률(91)(%)',
       '시장평균_미국(달러)(통화대원)', '소비자물가지수 총지수(2020=100)', '생산자물가지수 총지수(2015=100)',
       '전산업생산지수(2020=100)', '국가별수출액 합계(천달러)', 'WTI선물_종가', '브렌트유선물_종가',
       '천연가스선물_종가', '금선물_종가', '은선물_종가', '구리선물_종가', 'MACD', 'RSI', 'Increasing']
Target_column = 'Increasing'

### (1) Min-Max 정규화

In [6]:
print("Min-Max Normalization method is selected.")
Scalar_transform = MinMaxScaler(feature_range=(-1, 1)).fit(data.loc[:, [Target_column]])
data.loc[:, [Target_column]] = Scalar_transform.transform(data.loc[:, [Target_column]])
Input_transform = MinMaxScaler().fit(data.loc[:, Input_columns])
data.loc[:, Input_columns] = Input_transform.transform(data.loc[:, Input_columns])

Min-Max Normalization method is selected.


### (2) Transform Dataset with Sliding Windows

In [7]:
def sliding_windows(data, Input_columns, Target_column, seq_length):
    x = []
    y = []

    data_y = np.array(data.loc[:, [Target_column]])
    data_x = np.array(data.loc[:, Input_columns])


    for i in range(len(data_y)-seq_length):
        _x = data_x[i:(i+seq_length)]
        _y = data_y[i+seq_length]
        x.append(_x)
        y.append(_y)

    # x = np.array(x)
    return np.array(x),np.array(y)

seq_length = 60
x, y = sliding_windows(data, Input_columns, Target_column, seq_length)
print(x.shape)
print(y.shape)

(3761, 60, 27)
(3761, 1)


### (3) Train Test Split

In [8]:
N_test = 300
train_x = x[:-N_test, :, :]
train_y = y[:-N_test, :]
test_x = x[-N_test:, :, :]
test_y = y[-N_test:, :]

## 4. Modeling

In [9]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

model = Sequential()
model.add(LSTM(64, activation='relu',  input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Reshape((64,1)))
model.add(LSTM(32, activation='relu'))
model.add(Reshape((32,1)))
model.add(SimpleRNN(32, activation='tanh'))
model.add(Reshape((32,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=adam, 
              loss='binary_crossentropy',
              metrics=['accuracy'])
          
          

/home/jongwook95.lee/miniforge3/envs/tf2_env/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2023-12-20 00:06:37.481513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 00:06:38.557757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6955 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1
2023-12-20 00:06:38.558348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 7369 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:04:00.0, compute capability: 6.1


#### - Train 과정 중에서 가장 Test accuracy가 좋은 모델 저장

In [10]:
best_model_path = './experiment'
os.makedirs(best_model_path, exist_ok = True)
os.makedirs(os.path.join(best_model_path, 'lstm_rnn_model'), exist_ok = True)

checkpoint_path = os.path.join(os.path.join(best_model_path, 'lstm_rnn_model'), 'model.ckpt')
checkpoint_dir = os.path.dirname(checkpoint_path)
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,monitor='val_accuracy',
                                                 mode='max',
                                                 save_best_only=True)]

### - 학습

In [11]:
model.fit(train_x, train_y, epochs=200, batch_size=64, validation_data=[test_x, test_y], callbacks=callbacks)


Epoch 1/200
55/55 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5251
Epoch 1: val_accuracy improved from -inf to 0.50667, saving model to ./experiment/lstm_rnn_model/model.ckpt
55/55 [==============================] - 13s 189ms/step - loss: 0.6923 - accuracy: 0.5251 - val_loss: 0.6944 - val_accuracy: 0.5067
Epoch 2/200
55/55 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.5281
Epoch 2: val_accuracy did not improve from 0.50667
55/55 [==============================] - 10s 179ms/step - loss: 0.6924 - accuracy: 0.5281 - val_loss: 0.6939 - val_accuracy: 0.5067
Epoch 3/200
55/55 [==============================] - ETA: 0s - loss: 0.6919 - accuracy: 0.5284
Epoch 3: val_accuracy did not improve from 0.50667
55/55 [==============================] - 10s 180ms/step - loss: 0.6919 - accuracy: 0.5284 - val_loss: 0.6935 - val_accuracy: 0.5067
Epoch 4/200
55/55 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.5287
Epoch 4: val_a

## 5. 성능평가

In [12]:
model.load_weights(checkpoint_path)

In [13]:
model.evaluate(test_x, test_y)

10/10 [==============================] - 0s 29ms/step - loss: 0.6944 - accuracy: 0.5067


[0.694381058216095, 0.5066666603088379]

## 6. 미래 예측

In [19]:
future_x = np.array(data.loc[:, Input_columns])[-seq_length:]
future_x = np.expand_dims(future_x, axis = 0)

In [29]:
if model.predict(future_x)[0,0,0] > 0.5:
    print("Increase")
else:
    print("Decrease")

1/1 [==============================] - 0s 46ms/step
Increase
